**Connect google drive**

In [1]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Mon Sep 12 07:25:01 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
'1st attachment report-E022-01-0730-2017.docx'	 myCV.pdf
'2nd attachment report-E022-01-0730-2017.docx'	'My Drive'
 3RWorksheet-1.gdoc				'New folder (2)'
 ALX						'New folder (3)'
'attachment_1 (3).pdf'				'New folder (4)'
'Bouncy sample.mp3'				'New folder (5)'
 Classroom					'New folder (6)'
'Colab Notebooks'				 Oscar_Makumba_cv.pdf
 cucu						 oscarmomanyi_CV.pdf
 data2.gsheet					 pseudocode.gdoc
'Digital Twin _ 3DPrinter.docx'			'running sum.gdoc'
'Don type 138 bpm.wav'				 siemens.rar
'E022-01-0730 (1).2017assignment1.pdf'		 task1.gdoc
 E022-01-0730.2017assignment1.pdf		'Tell Tales 95bpm.wav'
'Getting started.pdf'				'Untitled document.gdoc'
'growth mindset.gdoc'				 yolov3


**1) Clone the Darknet**



In [3]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15490, done.
remote: Total 15490 (delta 0), reused 0 (delta 0), pack-reused 15490
Receiving objects: 100% (15490/15490), 14.06 MiB | 17.46 MiB/s, done.
Resolving deltas: 100% (10411/10411), done.


**2) Compile Darknet using Nvidia GPU**


In [4]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
           

# New Section

**3) Configure Darknet network for training YOLO V3**

In [6]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [7]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [8]:
# Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/yolov3_"

In [9]:
!echo "hole"> data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

In [10]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2022-09-12 07:30:36--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  17.0MB/s    in 10s     

2022-09-12 07:30:47 (15.1 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [11]:
!unzip /mydrive/yolov3/images.zip -d data/obj

Archive:  /mydrive/yolov3/images.zip
  inflating: data/obj/05.txt         
  inflating: data/obj/06.jpg         
  inflating: data/obj/06.txt         
  inflating: data/obj/07.jpeg        
  inflating: data/obj/07.txt         
  inflating: data/obj/08.jpg         
  inflating: data/obj/08.txt         
  inflating: data/obj/09.jpg         
  inflating: data/obj/10.jpg         
  inflating: data/obj/10.txt         
  inflating: data/obj/11.jpg         
  inflating: data/obj/11.txt         
  inflating: data/obj/12.jpg         
  inflating: data/obj/12.txt         
  inflating: data/obj/13.jpg         
  inflating: data/obj/13.txt         
  inflating: data/obj/14.jpg         
  inflating: data/obj/14.txt         
  inflating: data/obj/15.jpg         
  inflating: data/obj/15.txt         
  inflating: data/obj/16.jpeg        
  inflating: data/obj/16.txt         
  inflating: data/obj/17.jpg         
  inflating: data/obj/17.txt         
  inflating: data/obj/18.jpeg        
  inflating: 

In [12]:
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

15 0.156827 0.677419 0.298893 0.354839

['15', '0.156827', '0.677419', '0.298893', '0.354839']
0 data/obj/41.txt
0 0.156827 0.677419 0.298893 0.354839
15 0.244424 0.698006 0.050186 0.039886

['15', '0.244424', '0.698006', '0.050186', '0.039886']
1 data/obj/10.txt
0 0.244424 0.698006 0.050186 0.039886
15 0.314126 0.769231 0.059480 0.045584

['15', '0.314126', '0.769231', '0.059480', '0.045584']
1 data/obj/10.txt
0 0.314126 0.769231 0.059480 0.045584
15 0.519517 0.544160 0.053903 0.045584

['15', '0.519517', '0.544160', '0.053903', '0.045584']
1 data/obj/10.txt
0 0.519517 0.544160 0.053903 0.045584
15 0.357807 0.542735 0.035316 0.025641

['15', '0.357807', '0.542735', '0.035316', '0.025641']
1 data/obj/10.txt
0 0.357807 0.542735 0.035316 0.025641
15 0.513011 0.425926 0.044610 0.031339

['15', '0.513011', '0.425926', '0.044610', '0.031339']
1 data/obj/10.txt
0 0.513011 0.425926 0.044610 0.031339
15 0.459544 0.528651 0.018672 0.018484

['15', '0.459544', '0.528651', '0.018672', '0.018484']

In [13]:
import glob
images_list = glob.glob("data/obj/*.jpg")
print(images_list)

['data/obj/10.jpg', 'data/obj/09.jpg', 'data/obj/11.jpg', 'data/obj/21.jpg', 'data/obj/26.jpg', 'data/obj/12.jpg', 'data/obj/17.jpg', 'data/obj/29.jpg', 'data/obj/15.jpg', 'data/obj/20.jpg', 'data/obj/06.jpg', 'data/obj/42.jpg', 'data/obj/19.jpg', 'data/obj/28.jpg', 'data/obj/22.jpg', 'data/obj/01.jpg', 'data/obj/14.jpg', 'data/obj/08.jpg', 'data/obj/32.jpg', 'data/obj/13.jpg', 'data/obj/05.jpg', 'data/obj/24.jpg', 'data/obj/31.jpg', 'data/obj/39.jpg', 'data/obj/43.jpg']


In [14]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

**6) Start the training**

In [15]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

Streaming output truncated to the last 5000 lines.
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.408344), count: 3, class_loss = 0.861468, iou_loss = 1.109064, total_loss = 1.970533 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.416913), count: 17, class_loss = 4.244844, iou_loss = 6.184951, total_loss = 10.429795 
 total_bbox = 110056, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.000000), count: 1, class_loss = 0.003970, iou_loss = 0.000000, total_loss = 0.003970 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.537576), count: 6, class_loss = 1.716283, iou_loss = 1.780862, total_loss = 3.497145 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.414236), count: 18, class_loss = 4.228990, iou_loss = 8.262819, total_loss = 12.491810 
 total_bbox = 110080, rewritten_bbox = 0.000000 % 
v